In [4]:
import os

from utils_db import num_files_in_directory
from utils_db import dict_save_json, dict_load_json, load_jsonl
from utils_db import mkdir_if_not_exist
from template import Settings

class ExampleClass:

    def __init__(self, path_dir_database):
        # === save input(s) ===#
        self.path_dir_database = os.path.join(path_dir_database, 'tmp')
        
        # === variables === #
        
        
        
        mkdir_if_not_exist(self.path_dir_database)

        self.settings = Settings(self.path_dir_database)
        
        # === process === #
                        
        self.nr_wiki_pages = self.settings.get_item('nr_wikipedia_pages')
        
        # === create database === #
        self.flag_function_call(function_name = 'empty_function', arg_list = [])
    
    
    def empty_function(self):
        # description : 
        # input :
        # - 
        # output :
        # - 
        
        print('empty function')
        
    def get_item():
        if input_type == 'id':
            if output_type == 'title':
                return self.id_2_title_db.get_item(input_value)
            else:
                raise ValueError('output_type not in options', output_type)
        else:
            raise ValueError('input_type not in options', input_type)
    
    # === recurrent functions == #
    def flag_function_call(self, function_name, arg_list):
        check_flag = self.settings.check_function_flag(function_name, 'check')

        if check_flag == 'finished_correctly':
            return True

        elif check_flag == 'not_started_yet':
            self.settings.check_function_flag(function_name, 'start')
            
            values = getattr(self, function_name)(*arg_list)

            self.settings.check_function_flag(function_name, 'finish')
        else:
            raise ValueError('check_flag not in options', check_flag)


In [ ]:
pip3 install -r requirements.txt 

In [5]:
import os

from utils_db import dict_load_json, dict_save_json

class Settings:
    def __init__(self, path_settings_dir, file_name = 'settings'):
        
        self.path_settings = os.path.join(path_settings_dir, file_name + '.json')
        
        if os.path.isfile(self.path_settings):
            self.settings_dict = dict_load_json(self.path_settings)
        else:
            self.settings_dict = {}
            self.save_settings()
    
    def check_function_flag(self, function_name, status):
        # description: 
        # input:
        # - function_name : name of function that we are going to run
        # - status: 'start' (a flag is saved which indicates that a function has started),
        #           'check' (check if a function has started or finished correctly, else an error is raised)
        #           'finish' (set a flag which indicates that a function has finished correctly)
        
        if status == 'start':
            if 'flag_function_status' not in self.settings_dict:
                self.settings_dict['flag_function_status'] = {}
                
            if function_name in self.settings_dict['flag_function_status']:
                raise ValueError('cannot start a flag if it already exists', self.settings_dict['flag_function_status'][function_name])
            else:
                self.settings_dict['flag_function_status'][function_name] = False
                self.save_settings()

        elif status == 'check':
            # the three options are: (1) return 'not_started_yet', (2) return 'finished_correctly', (3) raise ValueError
            if 'flag_function_status' not in self.settings_dict:
                return 'not_started_yet'
            else:
                if function_name not in self.settings_dict['flag_function_status']:
                    return 'not_started_yet'
                else:
                    if self.settings_dict['flag_function_status'][function_name] == True:
                        return 'finished_correctly'
                    elif self.settings_dict['flag_function_status'][function_name] == False:
                        raise valueError('function was terminated too early', function_name)
                    else:
                        raise ValueError('the stored value should be True or False', self.settings_dict['flag_function_status'][function_name])

        elif status == 'finish':
            if 'flag_function_status' not in self.settings_dict:
                raise ValueError('cannot call the finish status if a function has not started yet')
            else:
                if function_name not in self.settings_dict['flag_function_status']:
                    raise ValueError('cannot call the finish status if a function has not started yet')
                else:
                    if self.settings_dict['flag_function_status'][function_name] == False:
                        self.settings_dict['flag_function_status'][function_name] = True
                        self.save_settings()
                    else:
                        raise ValueError('flag status should be started (False), but that is not the case', self.settings_dict['flag_function_status'][function_name])
        else:
            raise ValueError('status not in options', status)
    
    def add_item(self, key, value):
        self.settings_dict[key] = value
        self.save_settings()
    
    def get_item(self, key):
        return self.settings_dict[key]
    
    def save_settings(self):
        dict_save_json(self.settings_dict, self.path_settings)     
        